<a href="https://colab.research.google.com/github/ElFosco/NLP_assignments/blob/Fosco/Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
#!pip install gensim==3.6.0
import os, shutil  # file management
import sys  # system
import pandas as pd  # dataframe management
import numpy as np  # data manipulation
from tqdm import tqdm  # useful during debugging (progress bars)
from typing import List, Callable, Dict  # typing
import re  # regex
import urllib.request  # download files
import zipfile  # unzip files
import gensim  # embeddings
import gensim.downloader as gloader  # embeddings
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  # one-hot encoding
from matplotlib import pyplot as plt  # Plots
import nltk
from nltk.corpus import stopwords  # Remove stopwords
from nltk.stem import SnowballStemmer  # Stemming
from nltk.stem import WordNetLemmatizer

# Models
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import Bidirectional, Dense, SimpleRNN,GlobalAveragePooling1D,Flatten, Concatenate, Add, Average, Dot, Dropout
from keras.layers import concatenate, add, average, dot
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras import Input, Model
from keras.regularizers import l2

# F1
from sklearn.metrics import f1_score, accuracy_score, classification_report
from functools import partial

# Grid search
from sklearn.model_selection import GridSearchCV
import copy

#split
from sklearn.model_selection import train_test_split

In [ ]:
import os
import requests
import zipfile

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

def download_data(data_path):
    toy_data_path = os.path.join(data_path, 'fever_data.zip')
    toy_data_url_id = "1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1"
    toy_url = "https://docs.google.com/uc?export=download"

    if not os.path.exists(data_path):
        os.makedirs(data_path)

    if not os.path.exists(toy_data_path):
        print("Downloading FEVER data splits...")
        with requests.Session() as current_session:
            response = current_session.get(toy_url,
                                   params={'id': toy_data_url_id},
                                   stream=True)
        save_response_content(response, toy_data_path)
        print("Download completed!")

        print("Extracting dataset...")
        with zipfile.ZipFile(toy_data_path) as loaded_zip:
            loaded_zip.extractall(data_path)
        print("Extraction completed!")

download_data('dataset')

# Dataset Pre-processing

In [ ]:
try:
    STOPWORDS = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords')
    STOPWORDS = set(stopwords.words('english'))

STEMMER = SnowballStemmer("english")

nltk.download('wordnet') 
LEMMATIZER = WordNetLemmatizer()


def clean_text(text):

  #delete multiple quotes
  delete_multiple_quotes = "''|``|\.\."
  ris = re.sub(delete_multiple_quotes, '', text)

  #get only the sentence, delete the number before it and the keywords after it
  start_symbol = "^[0-9]*\\t"
  end_symbol = "( )?[\.|\?|\!|\,]( )?(\\t.*)?$"
  ris = re.sub(start_symbol, '', ris)
  ris = re.sub(end_symbol, '', ris)

  #convert the brackets into token, done for the claim string
  ris = re.sub("\(", " -LRB- ",ris)
  ris = re.sub("\)", " -RRB- ",ris)

  #check if numbers are present between tokens LSB and RSB, if it's not the case delete the content
  delete_content_lsb = "-LRB-(.[^0-9]*)-RRB-"
  ris = re.sub(delete_content_lsb, '', ris)

  #check if numbers are present in brackets, if it's not the case delete the content
  delete_content_brackets = "-LSB-(.[^0-9]*)-RSB-"
  ris = re.sub(delete_content_brackets, '', ris)

  #delete brackets token
  delete_brackets = "-LRB-|-RRB-|-RSB-|-LSB-"
  ris = re.sub(delete_brackets, ' ', ris)

  #deal with the &
  ris = re.sub("\&", ' and ', ris)

  #deal with the *
  ris = re.sub("star * reach", 'star*reach', ris)

  #remove tokens that we are not interested in
  remove_tokens = "[\-\"?!#`\$]"  # |[\.] " # added $ and \. handled alone`
  ris = re.sub(remove_tokens, ' ', ris)

  #delete additional spaces and last space
  #remove_spaces = "[ ]+"
  #remove_last_spaces= " $"
  #ris = re.sub(remove_spaces, ' ', ris)
  #ris = re.sub(remove_last_spaces, '', ris)

  #ris = ' '.join([x for x in ris.split() if x and x not in STOPWORDS])
  ris.strip()

  ris = " ".join([LEMMATIZER.lemmatize(word) for word in ris.split()])

  #handle ' (?)

  return ris.lower()


train_df = pd.read_csv('dataset/train_pairs.csv')
train_df = train_df.drop(['Unnamed: 0'],axis=1)
train_df['Evidence'] = train_df.apply(lambda row : clean_text(row['Evidence']), axis = 1)
train_df['Claim'] = train_df.apply(lambda row : clean_text(row['Claim']), axis = 1)


test_df = pd.read_csv('dataset/test_pairs.csv')
test_df = test_df.drop(['Unnamed: 0'],axis=1)
test_df['Evidence'] = test_df.apply(lambda row : clean_text(row['Evidence']), axis = 1)
test_df['Claim'] = test_df.apply(lambda row : clean_text(row['Claim']), axis = 1)

valid_df = pd.read_csv('dataset/val_pairs.csv')
valid_df = valid_df.drop(['Unnamed: 0'],axis=1)
valid_df['Evidence'] = valid_df.apply(lambda row : clean_text(row['Evidence']), axis = 1)
valid_df['Claim'] = valid_df.apply(lambda row : clean_text(row['Claim']), axis = 1)

In [ ]:
X_train = train_df.drop(['Label','ID'], axis=1)
X_val = valid_df.drop(['Label','ID'], axis=1)
X_test = test_df.drop(['Label','ID'], axis=1)

y_train = train_df['Label']
y_val = valid_df['Label']
y_test = test_df['Label']

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)


## Create GloVe embeddings (keep attention to the size, maybe we have to change it)

In [ ]:
def load_embedding_model(model_type: str,
                         embedding_dimension: int = 50) -> gensim.models.keyedvectors.KeyedVectors:
    """
    Loads a pre-trained word embedding model via gensim library.

    :param model_type: name of the word embedding model to load.
    :param embedding_dimension: size of the embedding space to consider

    :return
        - pre-trained word embedding model (gensim KeyedVectors object)
    """

    download_path = ""

    # Find the correct embedding model name
    if model_type.strip().lower() == 'word2vec':
        download_path = "word2vec-google-news-300"

    elif model_type.strip().lower() == 'glove':
        download_path = "glove-wiki-gigaword-{}".format(embedding_dimension)
    elif model_type.strip().lower() == 'fasttext':
        download_path = "fasttext-wiki-news-subwords-300"
    else:
        raise AttributeError("Unsupported embedding model type! Available ones: word2vec, glove, fasttext")

    # Check download
    try:
        emb_model = gloader.load(download_path)
    except ValueError as e:
        print("Invalid embedding model name! Check the embedding dimension:")
        print("Word2Vec: 300")
        print("Glove: 50, 100, 200, 300")
        raise e

    return emb_model

In [ ]:
def check_OOV_terms(embedding_vocabulary: List[str],
                    word_listing: List[str]):
    """
    Checks differences between pre-trained embedding model vocabulary
    and dataset specific vocabulary in order to highlight out-of-vocabulary terms.

    :param embedding_vocabulary: pre-trained word embedding model vocab (list)
    :param word_listing: dataset specific vocabulary (list)

    :return
        - list of OOV terms
    """
    
    oov = set(word_listing).difference(embedding_vocabulary)
    return list(oov)

In [ ]:
def build_embedding_matrix(embedding_model: gensim.models.keyedvectors.KeyedVectors,
                           embedding_dimension: int,
                           word_to_idx: Dict[str, int],
                           vocab_size: int,
                           oov_terms: List[str]) -> np.ndarray:
    """
    Builds the embedding matrix of a specific dataset given a pre-trained word embedding model

    :param embedding_model: pre-trained word embedding model (gensim wrapper)
    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param vocab_size: size of the vocabulary
    :param oov_terms: list of OOV terms (list)

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dimension), dtype=np.float32)

    for word, idx in tqdm(word_to_idx.items()):
        try:
            embedding_vector = embedding_model[word]
        except (KeyError, TypeError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector

    return embedding_matrix


def update_embedding_matrix(embedding_model: np.ndarray, 
                            embedding_dimension: int,
                            word_to_idx: Dict[str, int],
                            vocab_size: int,
                            oov_terms: List[str]) -> np.ndarray:
    """
    Builds the embedding matrix of a specific dataset given a pre-trained emdedding matrix

    :param embedding_model: pre-trained embedding matrix
    :param word_to_idx: vocabulary map (word -> index) (dict)
    :param vocab_size: size of the vocabulary
    :param oov_terms: list of OOV terms (list)

    :return
        - embedding matrix that assigns a high dimensional vector to each word in the dataset specific vocabulary (shape |V| x d)
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dimension), dtype=np.float32)

    for word, idx in tqdm(word_to_idx.items()):
        try:
            embedding_vector = embedding_model[idx]
        except (TypeError, IndexError):
            embedding_vector = np.random.uniform(low=-0.05, high=0.05, size=embedding_dimension)

        embedding_matrix[idx] = embedding_vector

    return embedding_matrix


## Tokenizer

In [ ]:
class KerasTokenizer(object):
    """
    A simple high-level wrapper for the Keras tokenizer.
    """

    def __init__(self, build_embedding_matrix=False, embedding_dimension=None,
                 embedding_model_type=None, tokenizer_args=None, embedding_model=None):
        if build_embedding_matrix:
            assert embedding_model_type is not None
            assert embedding_dimension is not None and type(embedding_dimension) == int

        self.build_embedding_matrix = build_embedding_matrix
        self.embedding_dimension = embedding_dimension
        self.embedding_model_type = embedding_model_type
        self.embedding_model = embedding_model
        self.embedding_matrix = None
        self.vocab = None

        tokenizer_args = {} if tokenizer_args is None else tokenizer_args
        assert isinstance(tokenizer_args, dict) or isinstance(tokenizer_args, collections.OrderedDict)

        self.tokenizer_args = tokenizer_args

    def build_vocab(self, data, **kwargs):
        print('Fitting tokenizer...')
        self.tokenizer = tf.keras.preprocessing.text.Tokenizer(**self.tokenizer_args)
        self.tokenizer.fit_on_texts(data)
        print('Fit completed!')

        self.vocab = self.tokenizer.word_index

        if self.build_embedding_matrix:
            if self.embedding_model is None:
              print('Loading embedding model! It may take a while...')
              self.embedding_model = load_embedding_model(model_type=self.embedding_model_type, 
                                                          embedding_dimension=self.embedding_dimension)
            
            print('Checking OOV terms in train...')
            self.oov_terms_train = check_OOV_terms(embedding_vocabulary=set(self.embedding_model.vocab.keys()),
                                             word_listing=list(self.vocab.keys()))
            
            print("Total OOV terms: {0} ({1:.2f}%)".format(len(self.oov_terms_train), 100*float(len(self.oov_terms_train)) / len(self.vocab)))

            print('Building the embedding matrix for train...')
            self.embedding_matrix = build_embedding_matrix(embedding_model=self.embedding_model,
                                                           word_to_idx=self.vocab,
                                                           vocab_size=len(self.vocab)+1,          
                                                           embedding_dimension=self.embedding_dimension,
                                                           oov_terms=self.oov_terms_train)
            print('Done for train!')

    def update_vocab(self, data, **kwargs):
      self.tokenizer.fit_on_texts(data)
      if self.build_embedding_matrix:
        old_vocab = self.vocab
        self.vocab = self.tokenizer.word_index
        print('Checking OOV terms...')
        self.oov_terms = check_OOV_terms(embedding_vocabulary=set(old_vocab.keys()), 
                                         word_listing=list(self.vocab.keys()))
        
        print("Total OOV terms: {0} ({1:.2f}%)".format(len(self.oov_terms), 100*float(len(self.oov_terms)) / len(self.vocab)))

        print('Building the embedding matrix...')
        self.embedding_matrix = update_embedding_matrix(embedding_model=self.embedding_matrix,
                                                       word_to_idx=self.vocab,
                                                       vocab_size=len(self.vocab)+1,          
                                                       embedding_dimension=self.embedding_dimension,
                                                       oov_terms=self.oov_terms)

    def get_info(self):
        return {
            'build_embedding_matrix': self.build_embedding_matrix,
            'embedding_dimension': self.embedding_dimension,
            'embedding_model_type': self.embedding_model_type,
            'embedding_matrix': self.embedding_matrix.shape if self.embedding_matrix is not None else self.embedding_matrix,
            'embedding_model': self.embedding_model,
            'vocab_size': len(self.vocab) + 1,
        }

    def tokenize(self, text):
        return text

    def convert_tokens_to_ids(self, tokens):
        if type(tokens) == str:
            return self.tokenizer.texts_to_sequences([tokens])[0]
        else:
            return self.tokenizer.texts_to_sequences(tokens)

    def convert_ids_to_tokens(self, ids):
        return self.tokenizer.sequences_to_texts(ids)

### Downloading embeddings

In [ ]:
embedding_dimension = 50
embedding_model = load_embedding_model(model_type="glove", 
                                       embedding_dimension=embedding_dimension)

### Creating tokenizer

In [ ]:
tokenizer_args = {
    'oov_token': "OOV_TOKEN",  # The vocabulary id for unknown terms during text conversion
    'lower' : True,  # default
    'filters' : '' 
}

tokenizer = KerasTokenizer(tokenizer_args=tokenizer_args,
                           build_embedding_matrix=True,
                           embedding_dimension=embedding_dimension,
                           embedding_model_type="glove", 
                           embedding_model=embedding_model)
tokenizer.build_vocab(X_train["Evidence"])
tokenizer.update_vocab(X_train["Claim"])

tokenizer_info = tokenizer.get_info()

print('Tokenizer info: ', tokenizer_info)

### Updating tokenizer with validation and test

In [ ]:
tokenizer.update_vocab(X_val["Claim"])
tokenizer.update_vocab(X_test["Claim"])
tokenizer.update_vocab(X_val["Evidence"])
tokenizer.update_vocab(X_test["Evidence"])

In [ ]:
a = list(tokenizer.vocab.keys())
a.sort()
print(a)

### Padding for x and computation of max sequence length

In [ ]:
def convert_text(df, tokenizer, is_training=False, max_seq_length=None):
    """
    Converts input text sequences using a given tokenizer

    :param texts: either a list or numpy ndarray of strings
    :tokenizer: an instantiated tokenizer
    :is_training: whether input texts are from the training split or not
    :max_seq_length: the max token sequence previously computed with
    training texts.

    :return
        text_ids: a nested list on token indices
        max_seq_length: the max token sequence previously computed with
        training texts.
    """


    text_ids_claim = tokenizer.convert_tokens_to_ids(df['Claim'])
    text_ids_evidence = tokenizer.convert_tokens_to_ids(df['Evidence'])

    # Padding
    if is_training:
        max_seq_length_claim = int(np.quantile([len(seq) for seq in text_ids_claim], 0.99))
        max_seq_length_evidence = int(np.quantile([len(seq) for seq in text_ids_evidence], 0.99))

        if max_seq_length_claim > max_seq_length_evidence:
            max_seq_length = max_seq_length_claim
        else:
          max_seq_length = max_seq_length_evidence

    else:
        assert max_seq_length is not None

    claims = [seq + [0] * (max_seq_length - len(seq)) for seq in text_ids_claim]
    claims = np.array([seq[:max_seq_length] for seq in claims])
    
    evidences = [seq + [0] * (max_seq_length - len(seq)) for seq in text_ids_evidence]
    evidences = np.array([seq[:max_seq_length] for seq in evidences])

    return max_seq_length, np.array([claims, evidences])
        

max_seq_length, x_train = convert_text(X_train, tokenizer, True)
print("Max token sequence: {}".format(max_seq_length))
print('X train shape: ', x_train.shape)

_, x_val = convert_text(X_val, tokenizer, False, max_seq_length)
print('X val shape: ', x_val.shape)

_, x_test = convert_text(X_test, tokenizer, False, max_seq_length)
print('X test shape: ', x_test.shape)

## Sentence embedding

---



In [ ]:
embedding_vector_length = embedding_dimension

In [ ]:
def firstModel(embedding_vector_length, dim):

  input = Input(shape=(max_seq_length))
  x = Embedding(len(tokenizer.vocab.keys())+1, embedding_vector_length, 
                      input_length=max_seq_length, 
                      trainable=True, 
                      mask_zero=True)(input)
  last_state = SimpleRNN(dim, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), 
                         bias_regularizer=l2(0.01), return_state=True)(x)
  
  RNN = Model(input, last_state, name="firstModel")

  return RNN

In [ ]:
# example to check if it is working, no train done
model_1 = firstModel(embedding_vector_length, 32)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      0.001,
      decay_steps=100000,
      decay_rate=0.96,
      staircase=True)
  
optim = Adam(learning_rate=lr_schedule)
model_1.compile(loss='binary_crossentropy', optimizer=optim, 
                   metrics=['accuracy'])
ris = model_1.predict(x_train[0][0].reshape(-1, len(x_train[0][0])))
print(X_train.Claim[0])
print(ris[1])

In [ ]:
def secondModel(embedding_vector_length, dim):
  
  input = Input(shape=(max_seq_length))
  x = Embedding(len(tokenizer.vocab.keys())+1, embedding_vector_length, 
                      input_length=max_seq_length, 
                      trainable=True, 
                      mask_zero=True)(input)
  states = SimpleRNN(dim, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), 
                     bias_regularizer=l2(0.01), return_sequences=True)(x)
  output = GlobalAveragePooling1D()(states)
  RNN = Model(input, output, name="secondModel")
  return RNN

In [ ]:
model_2 = secondModel(embedding_vector_length,32)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      0.001,
      decay_steps=100000,
      decay_rate=0.96,
      staircase=True)
  
optim = Adam(learning_rate=lr_schedule)
model_2.compile(loss='binary_crossentropy', optimizer=optim, 
                   metrics=['accuracy'])
ris = model_2.predict(x_train[0][0].reshape(-1, len(x_train[0][0])))
print(X_train.Claim[0])
print(ris[0])

In [ ]:
def thirdModel(embedding_vector_length, dim):
  
  MLP = Sequential()
  MLP.add(Embedding(len(tokenizer.vocab.keys())+1, embedding_vector_length, 
                      input_length=max_seq_length, 
                      trainable=True, 
                      mask_zero=True))
  MLP.add(Flatten())
  MLP.add(Dense(256, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(embedding_vector_length*max_seq_length,), activation='relu'))
  MLP.add(Dense(64, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='relu'))
  MLP.add(Dense(dim, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='sigmoid'))

  return MLP

In [ ]:
model_3 = thirdModel(embedding_vector_length, 32)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      0.001,
      decay_steps=100000,
      decay_rate=0.96,
      staircase=True)
  
optim = Adam(learning_rate=lr_schedule)
model_3.compile(loss='binary_crossentropy', optimizer=optim, 
                   metrics=['accuracy'])
ris = model_3.predict(x_train[0][0].reshape(-1, len(x_train[0][0])))
print(X_train.Claim[0])
print(ris[0])

In [ ]:
def fourthModel(embedding_vector_length, dim=-1):  # dim used for convenience
  
  EMB = Sequential()
  EMB.add(Embedding(len(tokenizer.vocab.keys())+1, embedding_vector_length, 
                      input_length=max_seq_length, 
                      trainable=True, 
                      mask_zero=True))
  EMB.add(GlobalAveragePooling1D())
  return EMB

In [ ]:
model_4 = fourthModel(embedding_vector_length)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      0.001,
      decay_steps=100000,
      decay_rate=0.96,
      staircase=True)
  
optim = Adam(learning_rate=lr_schedule)
model_4.compile(loss='binary_crossentropy', optimizer=optim, 
                   metrics=['accuracy'])
ris = model_4.predict(x_train[0][0].reshape(-1, len(x_train[0][0])))
print(X_train.Claim[0])
print(ris[0])

In [ ]:
# example to check if it is working, no train done
emb_claim = model_1.predict(x_train[0][0].reshape(-1, len(x_train[0][0])))[1]
emb_evidence = model_1.predict(x_train[1][0].reshape(-1, len(x_train[1][0])))[1]
# 1 case
first_emb = concatenate([emb_claim[0], emb_evidence[0]])
print(first_emb)
# 2 case
second_emb = add([emb_claim[0], emb_evidence[0]])
print(second_emb)
# 3 case
third_emb = average([emb_claim[0], emb_evidence[0]])
print(third_emb)

## Cosine Similarity

In [ ]:
model_4 = fourthModel(embedding_vector_length)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      0.001,
      decay_steps=100000,
      decay_rate=0.96,
      staircase=True)
  
optim = Adam(learning_rate=lr_schedule)
model_4.compile(loss='binary_crossentropy', optimizer=optim, 
                   metrics=['accuracy'])
emb_claim = model_4.predict(x_train[0][0].reshape(-1, len(x_train[0][0])))
emb_evidence = model_4.predict(x_train[1][0].reshape(-1, len(x_train[1][0])))

class_input = concatenate([emb_claim, emb_evidence])  # Concatenation as input
print("Initial classifier input shape: ", class_input.shape)

cos_sim = dot([emb_claim, emb_evidence], axes=1, normalize=True)  # cos similarity
print("\nCosine similarity: ", cos_sim)

class_input = concatenate([class_input, cos_sim])
print("\nFinal classifier input shape: ", class_input.shape)

## Model definition

In [ ]:
def Classifier(embed_model, model_type, type_merge, cosine_similarity,dense_size):
  input_c = Input(shape=(max_seq_length))
  input_e = Input(shape=(max_seq_length))
  embedding_c = embed_model(input_c)
  embedding_e = embed_model(input_e)

  if model_type == "firstModel":
    embedding_c = embedding_c[1]
    embedding_e = embedding_e[1]

  if type_merge == "concat":
      class_input = concatenate([embedding_c, embedding_e])
  elif type_merge == "sum":
      class_input = add([embedding_c, embedding_e])
  elif type_merge == "mean":
      class_input = average([embedding_c, embedding_e])

  if cosine_similarity:
      cos_sim = dot([embedding_c, embedding_e], axes=1, normalize=True)
      class_input = concatenate([class_input, cos_sim])
  x = Dropout(0.2)(class_input)
  x = Dense(dense_size, activation="relu", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
  output = Dense(1, activation="sigmoid")(x)

  return Model([input_c, input_e], output, name="Classifier")

## Evaluate models

In [ ]:
def evaluate_f1(model, x_data, y_data):
  predictions = model.predict(x_data)
  predictions = np.round(predictions)

  metrics = [
             accuracy_score,
             partial(f1_score, pos_label=1, average='macro')
             ]
  metric_names = [
      "accuracy",
      "f1-score"
  ]
  metric_info = evaluate_predictions(predictions=np.array(predictions),
                                    y=np.array(y_data),
                                    metrics=metrics,
                                    metric_names=metric_names)
  return metric_info

In [ ]:
def evaluate_predictions(predictions: np.ndarray,
                         y: np.ndarray,
                         metrics: List[Callable],
                         metric_names: List[str]):
    """
    Evaluates given model predictions on a list of metric functions

    :param predictions: model predictions in np.ndarray format
    :param y: ground-truth labels in np.ndarray format
    :param metrics: list of metric functions
    :param metric_names: list of metric names

    :return
        metric_info: dictionary containing metric values for each input metric
    """

    assert len(metrics) == len(metric_names)

    metric_info = {}

    for i, metric in enumerate(metrics):
        metric_name = metric_names[i]
        metric_value = metric(y_pred=predictions, y_true=y)
        metric_info[metric_name] = metric_value

    return metric_info

## Grid-search

In [ ]:
models = {'fourthModel': fourthModel}
parameters = {'epochs': range(10, 30, 10), 
              'batch_size':[ 512, 1024, 2048],
              'dim': [32],
              'merge_ops': ['mean', 'concat', 'sum'],  # pick the best one
              'cos_sim': [True, False],                # pick the best one
              'start_lr': [10**(-3),10**(-2)],
              'dense_size' : [32,256]
              }
best_scores = {1: 0, 2: 0}
best_params = {1: dict(), 2: dict()}

for epochs in parameters['epochs']:
  print("Epochs: ", epochs)
  for start_lr in parameters['start_lr']:
    print(" Start Learning Rate: ", start_lr)
    for batch_size in parameters['batch_size']:
      print("  Batch Size: ", batch_size)

      for dim in parameters['dim']:
        print("   Dim: ", dim)
        for model_name in models.keys():
          print("    Model: ", model_name)
          for merge_ops in parameters['merge_ops']:
            print("     Merge: ", merge_ops)
            for cos_sim in parameters['cos_sim']:
              print("      Cosine Similarity: ", cos_sim)
              for dense_size in parameters['dense_size']:
                print("       Dense size: ", dense_size)
                lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
                      start_lr,
                      decay_steps=100000,
                      decay_rate=0.96,
                      staircase=True)
                optim = Adam(learning_rate=lr_schedule)
                embed_model = models[model_name](embedding_vector_length, dim)
                model = Classifier(embed_model, model_name, merge_ops, 
                                  cosine_similarity=cos_sim,dense_size=dense_size)
                model.compile(loss='binary_crossentropy', optimizer=optim, 
                              metrics=['accuracy'])

                history = model.fit([x_train[0], x_train[1]], y_train, 
                          epochs=epochs, batch_size=batch_size, verbose=0)
                
                scores = evaluate_f1(model, [x_val[0], x_val[1]], y_val)
                print("     Scores: ", scores)
                if scores['f1-score'] > best_scores[2]:
                  if scores['f1-score'] > best_scores[1]:
                    best_scores[2] = best_scores[1]
                    best_scores[1] = scores['f1-score']
                    best_params[2] = best_params[1]
                    best_params[1] = {'epochs': epochs, 'batch_size': batch_size, 
                                      'dim': dim, 'start_lr': start_lr, 
                                      'model_name': model_name, 
                                      'merge_ops': merge_ops, 'cos_sim': cos_sim,
                                      'dense_size':dense_size}
                  else:
                    best_scores[2] = scores['f1-score']
                    best_params[2] = {'epochs': epochs, 'batch_size': batch_size, 
                                      'dim': dim, 'start_lr': start_lr, 
                                      'model_name': model_name, 
                                      'merge_ops': merge_ops, 'cos_sim': cos_sim,
                                      'dense_size':dense_size}
print(best_scores)
print(best_params)
                    

# Test

In [ ]:
models = {'firstModel': firstModel, 'secondModel': secondModel, 
          'thirdModel': thirdModel, 'fourthModel': fourthModel}
best_embed = 'firstModel'
best_epochs = 10
best_batch_size = 4096
best_dim = 32
best_start_lr = 0.01
best_merge_ops = 'concat'
best_cos_sim =  True
best_dense_size = 32


In [ ]:
models = {'firstModel': firstModel, 'secondModel': secondModel, 
          'thirdModel': thirdModel, 'fourthModel': fourthModel}

best_embed = best_params[1]['model_name']
best_epochs = best_params[1]['epochs']
best_batch_size = best_params[1]['batch_size']
best_dim = best_params[1]['dim']
best_start_lr = best_params[1]['start_lr']
best_merge_ops = best_params[1]['merge_ops']
best_cos_sim = best_params[1]['cos_sim']
best_dense_size = best_params[1]['dense_size']

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      best_start_lr,
      decay_steps=100000,
      decay_rate=0.96,
      staircase=True)
optim = Adam(learning_rate=lr_schedule)
embed_model = models[best_embed](embedding_vector_length, best_dim)
base_model = Classifier(embed_model, best_embed, best_merge_ops, 
                        cosine_similarity=best_cos_sim,dense_size=best_dense_size)
base_model.compile(loss='binary_crossentropy', optimizer=optim, 
                   metrics=['accuracy'])
history = base_model.fit(x=[x_train[0], x_train[1]], y=y_train, 
                         validation_data=([x_val[0], x_val[1]], y_val), 
                         epochs=best_epochs, batch_size=best_batch_size)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
predictions = base_model.predict([x_val[0], x_val[1]])
print(classification_report(y_val, np.round(predictions)))

In [ ]:
base_model.evaluate([x_test[0], x_test[1]], y_test)

In [ ]:
predictions = base_model.predict([x_test[0], x_test[1]])
print(classification_report(y_test, np.round(predictions)))

# Majority voting

In [ ]:
# A Multi input classification evaluation
def inputClassificationEvaluation(y, predictions):
  return classification_report(np.array(y), np.array(predictions), 
                               target_names=['refutes','supports'],
                               labels=[0,1]);

# B Claim verification evaluation
def claim_verification_evaluation(X, y, predictions):
  y_final=[]
  predictions_final=[]
  ris_label = -1
  ris_predicted = -1

  X['Label'] = y
  X['Predicted'] = predictions
  claims =  X.Claim.unique()

  for el in claims:
    # get every row with same Claim
    rows = X.loc[X['Claim'] == el]
    # get an array of the real label
    label = np.array(rows.Label)
    # get an array with the predictions
    predicted = np.array(rows.Predicted)
    # check the higher number of vote
    if sum(label) >= label.size/2:
      ris_label = 1
    else:
      ris_label = 0
    if sum(predicted) >=  predicted.size/2:
      ris_predicted = 1
    else:
      ris_predicted = 0
    # append to the final result 
    y_final.append(ris_label)
    predictions_final.append(ris_predicted)
  return inputClassificationEvaluation(y_final,predictions_final)

In [ ]:
predictions = base_model.predict([x_test[0], x_test[1]])
print(claim_verification_evaluation(X_test, y_test, np.round(predictions)))

# Assignment 2

**Due to**: 23/12/2021 (dd/mm/yyyy)

**Credits**: Andrea Galassi, Federico Ruggeri, Paolo Torroni

**Summary**: Fact checking, Neural Languange Inference (**NLI**)

# Intro

This assignment is centred on a particular and emerging NLP task, formally known as **fact checking** (or fake checking). As AI techniques become more and more powerful, reaching amazing results, such as image and text generation, it is more than ever necessary to build tools able to distinguish what is real from what is fake.

Here we focus on a small portion of the whole fact checking problem, which aims to determine whether a given statement (fact) conveys a trustworthy information or not. 

More precisely, given a set of evidences and a fact to verify, we would like our model to correctly predict whether the fact is true or fake.

In particular, we will see:

*   Dataset preparation (analysis and pre-processing)
*   Problem formulation: multi-input binary classification
*   Defining an evaluation method
*   Simple sentence embedding
*   Neural building blocks
*   Neural architecture extension

# The FEVER dataset

First of all, we need to choose a dataset. In this assignment we will rely on the [FEVER dataset](https://fever.ai).

The dataset is about facts taken from Wikipedia documents that have to be verified. In particular, facts could face manual modifications in order to define fake information or to give different formulations of the same concept.

The dataset consists of 185,445 claims manually verified against the introductory sections of Wikipedia pages and classified as ```Supported```, ```Refuted``` or ```NotEnoughInfo```. For the first two classes, systems and annotators need to also return the combination of sentences forming the necessary evidence supporting or refuting the claim.

## 2.1 Dataset structure

Relevant data is divided into two file types. Information concerning the fact to verify, its verdict and associated supporting/opposing statements are stored in **.jsonl** format. In particular, each JSON element is a python dictionary with the following relevant fields:

*    **ID**: ID associated to the fact to verify.

*    **Verifiable**: whether the fact has been verified or not: ```VERIFIABLE``` or ```NOT VERIFIABLE```.
    
*    **Label**: the final verdict on the fact to verify: ```SUPPORTS```, ```REFUTES``` or ```NOT ENOUGH INFO```.
    
*    **Claim**: the fact to verify.
    
*    **Evidence**: a nested list of document IDs along with the sentence ID that is associated to the fact to verify. In particular, each list element is a tuple of four elements: the first two are internal annotator IDs that can be safely ignored; the third term is the document ID (called URL) and the last one is the sentence number (ID) in the pointed document to consider.

**Some Examples**

---

**Verifiable**

```
{"id": 202314, "verifiable": "VERIFIABLE", "label": "REFUTES", "claim": "The New Jersey Turnpike has zero shoulders.", "evidence": [[[238335, 240393, "New_Jersey_Turnpike", 15]]]}
```

---

**Not Verifiable**

```
{"id": 113501, "verifiable": "NOT VERIFIABLE", "label": "NOT ENOUGH INFO", "claim": "Grease had bad reviews.", "evidence": [[[133128, null, null, null]]]}
```

---

## 2.2 Some simplifications and pre-processing

We are only interested in verifiable facts. Thus, we can filter out all non-verifiable claims.

Additionally, the current dataset format does not contain all necessary information for our classification purposes. In particular, we need to download Wikipedia documents and replace reported evidence IDs with the corresponding text.

Don't worry about that! We are providing you the already pre-processed dataset so that you can concentrate on the classification pipeline (pre-processing, model definition, evaluation and training).

You can download the zip file containing all set splits (train, validation and test) of the FEVER dataset by clicking on this [link](https://drive.google.com/file/d/1wArZhF9_SHW17WKNGeLmX-QTYw9Zscl1/view?usp=sharing). Alternatively, run the below code cell to automatically download it on this notebook.

**Note**: each dataset split is in .csv format. Feel free to inspect the whole dataset!

# Classification dataset

At this point, you should have a reay-to-go dataset! Note that the dataset format changed as well! In particular, we split the evidence set associated to each claim, in order to build `(claim, evidence)` pairs. The classification label is propagated as well.

We'll motivate this decision in the next section!

Just for clarity, here's an example of the pre-processed dataset:

---

**Claim**: "Wentworth Miller is yet to make his screenwriting debut."

**Evidence**: "2	He made his screenwriting debut with the 2013 thriller film Stoker .	Stoker	Stoker (film)"

**Label**: Refutes

---

[**Note**]: The dataset requires some text cleaning as you may have noticed!


# Problem formulation

As mentioned at the beginning of the assignment, we are going to formulate the fact checking problem as a binary classification task.

In particular, each dataset sample is comprised of:

*     A claim to verify
*     A set of semantically related statements (evidence set)
*     Fact checking label: either evidences support or refute the claim.

Handling the evidence set from the point of view of neural models may imply some additional complexity: if the evidence set is comprised of several sentences we might incur in memory problems.

To this end, we further simplify the problem by building (claim, evidence) pairs. The fact checking label is propagated as well.

Example:

     Claim: c1 
     Evidence set: [e1, e2, e3]
     Label: S (support)

--->

    (c1, e1, S),
    (c1, e2, S),
    (c1, e3, S)

## 4.1 Schema

The overall binary classification problem is summed up by the following (simplified) schema

![](https://drive.google.com/uc?export=view&id=1Wm_YBnFwgJtxcWEBpPbTBEVkpKaL08Jp)

Don't worry too much about the **Encoding** block for now. We'll give you some simple guidelines about its definition. For the moment, stick to the binary classification task definition where, in this case, we have 2 inputs: the claim to verify and one of its associated evidences.

# Architecture Guidelines

There are many neural architectures that follow the above schema. To avoid phenomena like the writer's block, in this section we are going to give you some implementation guidelines.

In particular, we would like you to test some implementations so that you explore basic approaches (neural baselines) and use them as building blocks for possible extensions.

## 5.1 Handling multiple inputs

The first thing to notice is that we are in a multi-input scenario. In particular, each sample is comprised of a fact and its asssociated evidence statement.

Each of these input is encoded as a sequence of tokens. In particular, we will have the following input matrices:

*    Claim: `[batch_size, max_tokens]`
*    Evidence: `[batch_size, max_tokens]`

Moreover, after the embedding layer, we'll have:

*    Claim: `[batch_size, max_tokens, embedding_dim]`
*    Evidence: `[batch_size, max_tokens, embedding_dim]`

But, we would like to have a 2D input to our classifier, since we have to give an answer at pair level. Therefore, for each sample, we would expect the following input shape to our classification block:

*   Classification input shape: `[batch_size, dim]`

**How to do that?**

We inherently need to reduce the token sequence to a single representation. This operation is formally known as **sentence embedding**. Indeed, we are trying to compress the information of a whole sequence into a single embedding vector.

Here are some simple solutions that we ask you to try out:

1.   Encode token sequences via a RNN and take the last state as the sentence embedding.

2.  Encode token sequences via a RNN and average all the output states.

3.  Encode token sequences via a simple MLP layer. In particular, if your input is a `[batch_size, max_tokens, embedding_dim]` tensor, the matrix multiplication works on the **max_tokens** dimension, resulting in a `[batch_size, embedding_dim]` 2D matrix. Alternatively, you can reshape the 3D input tensor from `[batch_size, max_tokens, embedding_dim]` to `[batch_size, max_tokens * embedding_dim]` and then apply the MLP layer.

4.   Compute the sentence embedding as the mean of its token embeddings (**bag of vectors**).

## 5.2 Merging multi-inputs

At this point, we have to think about **how** we should merge evidence and claim sentence embeddings.

For simplicity, we stick to simple merging strategies:

*     **Concatenation**: define the classification input as the concatenation of evidence and claim sentence embeddings

*     **Sum**: define the classification input as the sum of evidence and claim sentence embeddings

*     **Mean**: define the classification input as the mean of evidence and claim sentence embeddings

For clarity, if the sentence embedding of a single input has shape `[batch_size, embedding_dim]`, then the classification input has shape:

*     **Concatenation**: `[batch_size, 2 * embedding_dim]`

*     **Sum**: `[batch_size, embedding_dim]`

*     **Mean**: `[batch_size, embedding_dim]`

# Model evaluation

# A simple extension

Lastly, we ask you to modify previously defined neural architectures by adding an additional feature to the classification input.

We would like to see if some similarity information between the claim to verify and one of its associated evidence might be useful to the classification.

Compute the cosine similarity metric between the two sentence embeddings and concatenate the result to the classification input.

For clarity, since the cosine similarity of two vectors outputs a scalar value, the classification input shape is modified as follows:

*     **Concatenation**: `[batch_size, 2 * embedding_dim + 1]`

*     **Sum**: `[batch_size, embedding_dim + 1]`

*     **Mean**: `[batch_size, embedding_dim + 1]`



# Performance evaluation

Due to our simplifications, obtained results are not directly compatible with a traditional fact checking method that considers the evidence set as a whole.

Thus, we need to consider two types of evaluations.

---

A. **Multi-input classification evaluation**

This type of evaluation is the easiest and concerns computing evaluation metrics, such as accuracy, f1-score, recall and precision, of our pre-processed dataset.

In other words, we assess the performance of chosen classifiers.

---

B. **Claim verification evaluation**

However, if we want to give an answer concerning the claim itself, we need to consider the whole evidence set. 

Intuitively, for a given claim, we consider all its corresponding (claim, evidence) pairs and their corresponding classification outputs. 

At this point, all we need to do is to compute the final predicted claim label via majority voting.

---

Example:

    Claim: c1
    Evidence set: e1, e2, e3
    True label: S

    Pair outputs:
    (c1, e1) -> S (supports)
    (c1, e2) -> S (supports)
    (c1, e3) -> R (refutes)

    Majority voting:
    S -> 2 votes
    R -> 1 vote

    Final label:
    c1 -> S

Lastly, we have to compute classification metrics just like before.

Shortly speaking, implement both strategies for your classification metrics.

# Tips and Extras

## 8.1 Extensions are welcome!

Is this task too easy for you? Are you curious to try out things you have seen during lectures (e.g. attention)? Feel free to try everything you want!

**Don't forget to try neural baselines first!**

## 8.2 Comments and documentation

Remember to properly comment your code (it is not necessary to comment each single line) and don't forget to describe your work!

## 8.3 Organization

We suggest you to divide your work into sections. This allows you to build clean and modular code, as well as easy to read and to debug.

A possible schema:

*   Dataset pre-processing
*   Dataset conversion
*   Model definition
*   Training
*   Evaluation
*   Comments/Summary

# Evaluation

Which are the evaluation criteria on which we'll judge you and your work?

1. Pre-processing: whether you have done some pre-processing or not.
2. Sentence embedding: you should implement all required strategies (with an example and working code for each). That is, we, as evaluators, should be able to test all strategies without writing down new code.
3. Multiple inputs merging strategies: you should implement all required strategies (with an example and working code for each).
4. Similarity extension: you should implement the cosine similarity extension (with an example and working code).
5. Voting strategy: you should implement the majority voting strategy and provide results.
6. Report: when submitting your notebook, you should also attach a small summary report that describes what you have done (provide motivations as well for abitrary steps. For instance, "We've applied L2 regularization since the model was overfitting".

Extras (possible extra points):

1. Any well defined extension is welcome!
2. Well organized and commented code is as important as any other criteria.

# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Andrea Galassi -> a.galassi@unibo.it
* Federico Ruggeri -> federico.ruggeri6@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

*Note*: We highly recommend you to check the [course useful material](https://virtuale.unibo.it/pluginfile.php/1036039/mod_resource/content/2/NLP_Course_Useful_Material.pdf) for additional information before contacting us!

# FAQ

---

**Question**: Can I do something text pre-processing?

**Answer:** You have to! If you check text data, the majority of sentences need some cleaning.

---

**Question**: The model architecture schema is not so clear, are we doing end-to-end training?

**Answer**: Exactly! All models can be thought as:

1. Input
2. (word) Embedding
3. Sentence embedding
4. Multiple inputs merging
5. Classification

---

**Question**: Can I extend models by adding more layers?

**Answer**: Feel free to define model architectures as you wish, but remember satisfy our requirements. This assignment should not be thought as a competition to achieve the best performing model: fancy students that want to show off but miss required assignment objectives will be punished!!

---

**Question**: I'm struggling with the implementation. Can you help me?

**Answer**: Yes sure! Contact us and describe your issue. If you are looking for a particular type of operation, you can easily check the documentation of the deep learning framework you are using (google is your friend).

---

**Question**: Can I try other encoding strategies or neural architectures?

**Answer:** Absolutely! Remember to try out recommended neural baselines first and only then proceed with your extensions.

---

**Question**: Do we have to test all possible sentence embedding and input merging combinations?

**Answer**: Absolutely no! Feel free to pick one sentence embedding strategy and try all possible input merging strategies with it! For instance, pick the best performing sentence embedding method and proceed with next steps (extras included). Please, note that you still have to implement all mentioned strategies!

---

**Question**: I'm hitting out of memory error when training my models, do you have any suggestions?

**Answer**: Here are some common workarounds:

1. Try decreasing the mini-batch size
2. Try applying a different padding strategy (if you are applying padding): e.g. use quantiles instead of maximum sequence length
3. Check the efficiency of your custom code implementation (if any)
4. Try to define same length mini-batches to avoid padding (**It should not be necessary here!**)

---

**Question**: I'm hitting CUDNN_STATUS_BAD_PARAM error! What I'm doing wrong?

**Answer**: This error is a little bit tricky since the stack trace is not meaningful at all! This error occurs when the RNN is fed with a sequence of all 0s and pad masking is enabled (e.g. from the embedding layer). Please, check your conversion step, since there might be an error that leads to the encoding of a sentence to all 0s.

---